In [182]:
from scipy import stats as st

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

import plotly.express as px
from plotly import graph_objects as go

import re 
import math as mth

## Обзор данных

Посмотрим что находится в данных.

In [183]:
df = pd.read_csv("dataset.csv")

In [184]:
df

,task_rk,duratoin_sec,hit_status_result_desc,hid,using_flg,agent_login
0,196,"55,66","Дозвон, Перезвонить",1,NaN,1190lasa
1,630,0,Недозвон,4,NaN,zovadxta
2,630,0,Недозвон,4,NaN,lovaraka
3,852,189,"Дозвон, Перезвонить",1,NaN,7876nssh
4,1635,0,Недозвон,4,NaN,inovgaan
...,...,...,...,...,...,...
89169,99997999,0,Недозвон,1,NaN,9363vesh
89170,99998332,"103,88","Дозвон, Перезвонить",4,NaN,ayanyuyu
89171,99998332,0,Недозвон,4,NaN,vkiniyua
89172,99999355,"32,78","Дозвон, Отказ",4,NaN,oyansgat


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89174 entries, 0 to 89173
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   task_rk                 89174 non-null  int64  
 1   duratoin_sec            89174 non-null  object 
 2   hit_status_result_desc  89174 non-null  object 
 3   hid                     89174 non-null  int64  
 4   using_flg               2525 non-null   float64
 5   agent_login             89174 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 4.1+ MB


In [186]:
df.hit_status_result_desc.value_counts()

Недозвон                                  50461
Дозвон, Отказ                             18459
Дозвон, Перезвонить                       15108
Дозвон, Успешно                            3310
Дозвон, Отложить                           1680
Не было звонка                              130
Дозвон, Некорректное задание по звонку       26
Name: hit_status_result_desc, dtype: int64

---
## Подготовка данных

Заменим названия столбцов

In [187]:
df.columns = ["task_id", "duration_sec", "result", "product_id", "using", "agent_login"]
df.head(2)

,task_id,duration_sec,result,product_id,using,agent_login
0,196,"55,66","Дозвон, Перезвонить",1,NaN,1190lasa
1,630,0,Недозвон,4,NaN,zovadxta


Теперь названия столбцов стали понятнее
* `task_id` — идентификатор задания;
* `duratoin_sec` — продолжительность звонка;
* `result` — результата коммуникации;
* `product_id` — идентификатор продукта;
* `using` — флаг начала пользования продуктом;
* `agent_login` — логин оператора.

Заменим тип данных в `duration_sec`

In [188]:
# Функция заменяет ',' на '.' и переведет во float
def replace_comma(dur):
    nw = []
    for i in dur:
        i = float(i.replace(",","."))
        nw.append(i)
    return nw

In [189]:
df["duration_sec"] = replace_comma(df["duration_sec"])


Проверим пропуски

In [190]:
# подсчёт пропусков
df.isna().sum()

task_id             0
duration_sec        0
result              0
product_id          0
using           86649
agent_login         0
dtype: int64

В нашем случае, пропуски в `using` показывают что в клиент ещё не дошел до продукта, чтобы принять решение по его ипользованию.

Проверим дупликаты

In [191]:
print('Кол-во дубликатов =',df.duplicated(["task_id", "duration_sec", "result", "product_id", "using", "agent_login"]).sum())

Кол-во дубликатов = 402


*402 дубликатов, скорей всего эти дубликаты являются попытками оператора дозвонится до клиента и их можно спокойно удалить. Но сначала посмотрим что там:*

In [192]:
df[df.duplicated()==True].sort_values(by=["task_id", "duration_sec", "result", "product_id", "using", "agent_login"]).head(30)

,task_id,duration_sec,result,product_id,using,agent_login
231,62937,0.0,Недозвон,1,NaN,8843evod
593,169912,0.0,Недозвон,4,NaN,eevaanom
707,205167,0.0,Недозвон,4,NaN,kovantta
741,214203,0.0,Недозвон,1,NaN,2415aaty
837,248928,0.0,Недозвон,4,NaN,uluuinnu
881,265455,0.0,Недозвон,4,NaN,eevaanom
1824,587280,0.0,Недозвон,4,NaN,ryannaxa
1831,587651,0.0,Недозвон,4,NaN,6691agsa
1889,606326,0.0,Недозвон,2,NaN,5596avlo
1895,607704,0.0,Недозвон,4,NaN,2238aagr


In [193]:
df[(df["task_id"]==62937) & (df["result"]=="Недозвон")]

,task_id,duration_sec,result,product_id,using,agent_login
228,62937,0.0,Недозвон,1,NaN,kovaevsk
229,62937,0.0,Недозвон,1,NaN,8843evod
231,62937,0.0,Недозвон,1,NaN,8843evod
232,62937,0.0,Недозвон,1,NaN,tovaarbo


Видно что одни и те же действия повторяются. Удалим их:

In [194]:
df = df.drop_duplicates()
print('Кол-во оставшихся дубликатов =', df.duplicated().sum())

Кол-во оставшихся дубликатов = 0


Проверим:

In [195]:
print('Было строчек - 89174 /', "удалили дубликатов - 402", "Осталось строк -", (89174-402))

Было строчек - 89174 / удалили дубликатов - 402 Осталось строк - 88772


In [196]:
df = df.reset_index(drop=True)
df

,task_id,duration_sec,result,product_id,using,agent_login
0,196,55.66,"Дозвон, Перезвонить",1,NaN,1190lasa
1,630,0.00,Недозвон,4,NaN,zovadxta
2,630,0.00,Недозвон,4,NaN,lovaraka
3,852,189.00,"Дозвон, Перезвонить",1,NaN,7876nssh
4,1635,0.00,Недозвон,4,NaN,inovgaan
...,...,...,...,...,...,...
88767,99997999,0.00,Недозвон,1,NaN,9363vesh
88768,99998332,103.88,"Дозвон, Перезвонить",4,NaN,ayanyuyu
88769,99998332,0.00,Недозвон,4,NaN,vkiniyua
88770,99999355,32.78,"Дозвон, Отказ",4,NaN,oyansgat


*Всё сходится 88772 строк.*

*Вывод. Таблица готова для анализа.*

In [218]:
print('Кол-во успешных звонков =', df[df["duration_sec"]!=0]["duration_sec"].count())
sum = 0
for g in ["Дозвон, Отказ", "Дозвон, Перезвонить","Дозвон, Успешно","Дозвон, Отложить","Дозвон, Некорректное задание по звонку"]:
    users_gr = df[df["result"]==g]["result"].count()
    sum += users_gr
    print(f'Результат {g} — {users_gr} кол-во звонков')
print(f'При сложениии получается — {sum} дозвонов')

Кол-во успешных звонков = 38523
Результат Дозвон, Отказ — 18459 кол-во звонков
Результат Дозвон, Перезвонить — 15108 кол-во звонков
Результат Дозвон, Успешно — 3305 кол-во звонков
Результат Дозвон, Отложить — 1680 кол-во звонков
Результат Дозвон, Некорректное задание по звонку — 26 кол-во звонков
При сложениии получается — 38578 дозвонов


*Звонков с продолжительностью больше 0 меньше суммы всех дозвонов. Получается, что дозвоном считается звонок который клиент  отклонил*

In [220]:
df["result"].value_counts()

Недозвон                                  50064
Дозвон, Отказ                             18459
Дозвон, Перезвонить                       15108
Дозвон, Успешно                            3305
Дозвон, Отложить                           1680
Не было звонка                              130
Дозвон, Некорректное задание по звонку       26
Name: result, dtype: int64

In [223]:
df[(df["duration_sec"]==0) & (df["result"]=="Дозвон, Успешно")]

,task_id,duration_sec,result,product_id,using,agent_login
4166,1561170,0.0,"Дозвон, Успешно",4,0.0,cyanrser
4778,1755964,0.0,"Дозвон, Успешно",4,0.0,cyanrser
8747,3097008,0.0,"Дозвон, Успешно",4,0.0,cyanrser
8878,3139115,0.0,"Дозвон, Успешно",4,0.0,cyanrser
10804,3920798,0.0,"Дозвон, Успешно",4,0.0,cyanrser
11179,4121304,0.0,"Дозвон, Успешно",4,0.0,cyanrser
11202,4136287,0.0,"Дозвон, Успешно",4,0.0,cyanrser
11952,4491627,0.0,"Дозвон, Успешно",4,0.0,cyanrser
12209,4608502,0.0,"Дозвон, Успешно",4,0.0,cyanrser
13355,5163839,0.0,"Дозвон, Успешно",4,0.0,cyanrser
